In [326]:
from functools import reduce
import statistics as st
import cv2
import pytesseract
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from datetime import datetime
#import matplotlib.pyplot as plt
import os
#from PyPDF2 import PdfReader
#%matplotlib inline
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\epenago\AppData\Local\Tesseract-OCR\tesseract.exe'

In [327]:
# get grayscale image
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# noise removal
def remove_noise(image):
    return cv2.medianBlur(image,5)
 
#thresholding
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

#dilation
def dilate(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.dilate(image, kernel, iterations = 1)
    
#erosion
def erode(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.erode(image, kernel, iterations = 1)

#opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

#canny edge detection
def canny(image):
    return cv2.Canny(image, 100, 200)

In [328]:
def calcular_espacios(diccionario, indices):
    
    suma = 0
    bandera = True
  
    width_espacios = []
    for i in range(len(indices)):
             
        if(bandera == True):
            
            bandera = False
            suma = 0
            suma += int(diccionario["left"][indices[i]])+int(diccionario["width"][indices[i]])

            if i < len(indices)-1:
                if int(diccionario["top"][indices[i]]) >= int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]])+10 or int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) > int(diccionario["top"][indices[i+1]])-10:
                    pass
                else:
                    bandera = True
            else:           
                    bandera = True
                    

        elif(i < len(indices)-1):
            width_espacios.append(diccionario["left"][indices[i]] - suma)
            suma += int(diccionario["width"][indices[i]] + (diccionario["left"][indices[i]] - suma))
            if int(diccionario["top"][indices[i]]) >= int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]])+10 or int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) > int(diccionario["top"][indices[i+1]])-10:
                pass
            else:
                bandera = True
               

        elif(i <= len(indices)-1):
            width_espacios.append(diccionario["left"][indices[i]] - suma)
            suma += int(diccionario["width"][indices[i]] + (diccionario["left"][indices[i]] - suma))
            bandera = True
            
    #print(width_espacios)
    width_espacios.sort() 
    
    moda = st.mode(width_espacios)
    return (moda-(moda*1), moda+(moda*1))

In [329]:
def coordenadas_extraccion(diccionario, indices, intervalo):
    
    datos_extraidos = []
    dict = {}
    coordenadas = []
    suma = 0
    bandera = True
    indice_anterior = -1
    strings = []
    string = ""
    index = []
    line = 0

    for i in range(len(indices)):
             
        if(bandera == True):
            indice_anterior = i
            
            bandera = False
            suma = 0
            suma += int(diccionario["left"][indices[i]])+int(diccionario["width"][indices[i]])

            # Se guarda la palabra y su indice
            string = diccionario["text"][indices[i]]
            index.append(indices[i])
            
            if i < len(indices)-1:
                if (int(diccionario["top"][indices[i]]) >= int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]])+13 or int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) > int(diccionario["top"][indices[i+1]])-13):
                    
                    if (diccionario["left"][indices[i+1]] - suma) < intervalo[0] or (diccionario["left"][indices[i+1]] - suma) > intervalo[1]:
                        bandera = True
                        #print(indices[i])
                        coordenadas.append([diccionario["left"][indices[indice_anterior]], 
                        diccionario["top"][indices[indice_anterior]], 
                        suma,
                        diccionario["top"][indices[i]]+diccionario["height"][indices[i]]])
                        
                        dict["text"] = string
                        dict["line"] = line
                        dict["index"] = index
                        dict["coordinates"] = (diccionario["left"][indices[indice_anterior]], suma)
                        datos_extraidos.append(dict)
                                                                    
                        dict = {}                        
                        string = ""
                        index = []

                        bandera = True
                        indice_anterior = -1
                else:
                    bandera = True
                    #print(indices[i])
                    coordenadas.append([diccionario["left"][indices[indice_anterior]], 
                    diccionario["top"][indices[indice_anterior]], 
                    suma,
                    diccionario["top"][indices[i]]+diccionario["height"][indices[i]]])
                    

                    dict["text"] = string
                    dict["line"] = line
                    dict["index"] = index
                    dict["coordinates"] = (diccionario["left"][indices[indice_anterior]], suma)
                    datos_extraidos.append(dict)
                                                                
                    dict = {}                        
                    string = ""
                    index = []
                    line += 1

                    bandera = True
                    indice_anterior = -1
            else:           
                    coordenadas.append([diccionario["left"][indices[indice_anterior]], 
                    diccionario["top"][indices[indice_anterior]], 
                    suma,
                    diccionario["top"][indices[i]]+diccionario["height"][indices[i]]])

                    dict["text"] = string
                    dict["line"] = line
                    dict["index"] = index
                    dict["coordinates"] = (diccionario["left"][indices[indice_anterior]], suma)
                    datos_extraidos.append(dict)
                                                                
                    dict = {}                        
                    string = ""
                    index = []

                    bandera = True
                    indice_anterior = -1

        elif(i < len(indices)-1):
            string += (" " + diccionario["text"][indices[i]])
            index.append(indices[i])

            suma += int(diccionario["width"][indices[i]] + (diccionario["left"][indices[i]] - suma))
            if (int(diccionario["top"][indices[i]]) >= int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]])+13 or int(diccionario["top"][indices[i]]) < int(diccionario["top"][indices[i+1]]) and int(diccionario["top"][indices[i]]) > int(diccionario["top"][indices[i+1]])-13):
                
                if (diccionario["left"][indices[i+1]] - suma) < intervalo[0] or (diccionario["left"][indices[i+1]] - suma) > intervalo[1]:
                    bandera = True
                    #print(indices[i])
                    coordenadas.append([diccionario["left"][indices[indice_anterior]], 
                    diccionario["top"][indices[indice_anterior]], 
                    suma,
                    diccionario["top"][indices[i]]+diccionario["height"][indices[i]]])

                    dict["text"] = string
                    dict["line"] = line
                    dict["index"] = index
                    dict["coordinates"] = (diccionario["left"][indices[indice_anterior]], suma)
                    datos_extraidos.append(dict)
                                                                
                    dict = {}                        
                    string = ""
                    index = []

                    bandera = True
                    indice_anterior = -1
            else:
                bandera = True
                #print(indices[i])
                coordenadas.append([diccionario["left"][indices[indice_anterior]], 
                diccionario["top"][indices[indice_anterior]], 
                suma,
                diccionario["top"][indices[i]]+diccionario["height"][indices[i]]])

                dict["text"] = string
                dict["line"] = line
                dict["index"] = index
                dict["coordinates"] = (diccionario["left"][indices[indice_anterior]], suma)
                datos_extraidos.append(dict)
                                                            
                dict = {}                        
                string = ""
                index = []
                line += 1

                bandera = True
                indice_anterior = -1

        elif(i <= len(indices)-1):
            suma += int(diccionario["width"][indices[i]] + (diccionario["left"][indices[i]] - suma))
            coordenadas.append([diccionario["left"][indices[indice_anterior]], 
            diccionario["top"][indices[indice_anterior]], 
            suma,
            diccionario["top"][indices[i]]+diccionario["height"][indices[i]]])

            string = ""
            index = []

            bandera = True
            indice_anterior = -1

    return (coordenadas, datos_extraidos)

En el Metodo extraccion_columns logramos separar las columnas extrayedo y clasificando los indices en el dicionario data_ordenaizer

In [330]:
def extracion_colums(df):

    arr = np.array(df["coordinates"].to_list())
    colums = []
    
    for i in np.arange(len(arr)):
        temp = []
        temp.append(i)
        for j in np.arange(len(arr)):

            if i == j: continue
            interval1 = np.arange(arr[i,0], arr[i,1]+1, dtype=int)
            interval2 = np.arange(arr[j,0], arr[j,1]+1, dtype=int)

            sum = 0
            for k in interval1:
                if k in interval2:
                    sum += 1
    
            tamaño1 = len(interval1)
            tamaño2 = len(interval2)
            percent1 = int((100/tamaño1)*sum )
            percent2 = int((100/tamaño2)*sum )
            """
            print("tamaño1: ", tamaño1)
            print("tamaño2: ", tamaño2)
            print("la suma: ", sum)
            print("percent1: ", percent1, "%")
            print("percent2: ", percent2, "%")
            """
            if percent1 > 55 and percent2 > 55:
                temp.append(j)
        
        colums.append(temp)
        
    cl = []
    descartados = []
    for i in np.arange(len(colums)):
        temp = []
        temp = colums[i]
        if i in descartados: continue
        for j in np.arange(len(colums)):

            if i == j: continue
            interval1 = colums[i]
            interval2 = colums[j]

            sum = 0
            for k in interval1:
                if k in interval2:
                    sum += 1
    
            tamaño1 = len(interval1)
            tamaño2 = len(interval2)
            percent1 = int((100/tamaño1)*sum )
            percent2 = int((100/tamaño2)*sum )
            """
            print("tamaño1: ", tamaño1)
            print("tamaño2: ", tamaño2)
            print("la suma: ", sum)
            print("percent1: ", percent1, "%")
            print("percent2: ", percent2, "%")
            """
            if percent1 > 20 or percent2 > 20:
                temp += colums[j]
                descartados.append(j)
        cl.append(temp)

    colums = []    
    #se eliminan elementos repetidos en las columnas
    for element in cl:
        temp = []
        for i in element:
            if i not in temp:
                temp.append(i)
        colums.append(temp)
    

    print("el tamaño de cl es: ", len(cl))
    #en este bloque se saca la pocision en pixeles de la imagen
    min_max = []
    for element in colums:   
        menor = arr[element[0],0]
        mayor = arr[element[0],1]
        for i in element:
            if arr[i,0] < menor:
                menor = arr[i,0]
            
            if arr[i,1] > mayor:
                mayor = arr[i,1]
        min_max.append((menor,mayor))

    for i in range(len(min_max)):
        print(min_max[i], colums[i])

    cl = []
    descartados = []
    by_delete = []
    for i in range(len(min_max)):
        temp= []
        temp= colums[i]
        if i in descartados: continue
        for j in range(len(min_max)):

            if i==j: continue
            if min_max[i][0]-10 <= min_max[j][0] and min_max[i][1]+10 >= min_max[j][1]:
                temp += colums[j]
                descartados.append(j)
                if i > j:
                    by_delete.append(colums[j])
        cl.append(temp)

    colums = cl
    print("estos son a eliminar", by_delete)
    for k in by_delete:
        if k in colums:
            colums.remove(k)

    print()
    zises = []
    for element in colums:
        print(element)
        zises.append(len(element))
    
    mean = st.mean(zises)
    print("la media de los len ", mean)
    by_delete = []
    for i in range(len(colums)):
        percent20 = (mean/100)*20
        #print("percent20: ", percent20)
        if len(colums[i]) < percent20:
            by_delete.append(colums[i])

    for element in colums:
        print(element)
        print

    print("elementos a eliminar: ", by_delete)
    for k in by_delete:
        if k in colums:
            colums.remove(k)

    for element in colums:
        print(element)

    min_max_h= []
    for element in colums:   
        menor = arr[element[0],0]
        mayor = arr[element[0],1]
        for i in element:
            if arr[i,0] < menor:
                menor = arr[i,0]
            
            if arr[i,1] > mayor:
                mayor = arr[i,1]
        min_max_h.append((menor,mayor))

    min_max_h_unOrder = min_max_h.copy()
    min_max_h.sort()

    cl = []
    for i in np.arange(len(colums)):
        for j in np.arange(len(colums)):
            if min_max_h_unOrder[j] == min_max_h[i]:
                cl.append(colums[j])

    colums = cl

    print(min_max_h)
    
    return min_max_h,colums

El metodo build_df toma el arreglo con las columnas separadar y utiliza el diccionario construido en extracion_colums 
para crear un objeto tipo pandas.DataFrame con la misma distribucion de los datos que estaba en la imagen

In [331]:
def build_df(pd_dataOrdenizer, colums):
    temp = pd.DataFrame()
    pd_columns = pd.DataFrame()
    for i in range(len(colums)):
        temp = pd.DataFrame.copy(pd_dataOrdenizer.loc[colums[i], ["text", "line"]])
        temp = temp.drop_duplicates()
        temp = temp.sort_values("line")
        temp = pd.Series(temp["text"].tolist(), index=temp["line"].tolist())
        temp = temp.loc[~temp.index.duplicated(keep='first')]
        pd_columns = pd.concat([pd_columns, temp], axis=1, ignore_index=True)

    pd_columns = pd_columns.sort_index()
    return pd_columns

In [332]:
def del_headEnd(df):

    freq = df['line'].value_counts().sort_index().head(10)
    to_delete = []
    bandera = True

    for i in range(len(freq)):
        if bandera == True:
            if freq[i] == 1:
                to_delete.append(i)
            
            elif freq[i] == 2 and freq[i+1] == 1:
                to_delete.append(i)
            else: bandera = False
    
    for i in to_delete:
        df.drop(df.index[df['line'] == i] , axis=0, inplace=True)
    df.reset_index(inplace=True)
    df.drop(['level_0'], axis=1, inplace=True)
    

In [333]:
def isTime(string):
    try:
        date = datetime.strptime(string	, '%d/%m/%Y')
        return str(date.year)
    except ValueError:
        return np.nan


In [334]:
def isNumeric(s):
    try:
        if type(s) != float:
            s = s.replace('(','') 
            s = s.replace(')','') 
        float(s)
        return True
    except ValueError:
        return False

In [335]:
def convert_parenthesis(s):
   
    if type(s) != float:
        if '(' in s or ')' in s:
            s = s.replace('(','') 
            s = s.replace(')','')
            s = '-' + s
    
    return s


In [336]:
def isNumericColum(df): 
    type_index=[]                   
    str_index=[]
    counter_num = 1
    #to_delete = []

    for i in df.columns:
        size = df[i].apply(isNumeric).value_counts().sum()
        try:
            if 100*df[i].apply(isNumeric).value_counts()[True]/size >= 80:
                type_index.append('num')
                df[i] = df[i].apply(convert_parenthesis)
            else:
                type_index.append('text')
            
        except KeyError:
            type_index.append('text')

    years = np.arange(2018,datetime.now().year + 1)
    years = [str(i) for i in years]
    years.append('NOTAS')
    for i in df.columns:
        if type_index[i] == 'text':
            str_index.append(df[i][~df[i].isnull()].head(1).to_list()[0])
            #to_delete.append(df[i][~df[i].isnull()].head(1).index.to_list()[0])
        else:
            
            df_three = df[i][~df[i].isnull()].head(3).apply(lambda x: x if x.upper() in years else np.nan)
            if df_three.count() == 0:
                df_three = df[i][~df[i].isnull()].head(3).apply(isTime)
                if df_three.count() == 0:
                    str_index.append('num' + str(counter_num))
                    counter_num += 1
                else:
                    str_index.append(df_three[~df_three.isnull()].head(1).to_list()[0])
            else:
                str_index.append(df_three[~df_three.isnull()].head(1).to_list()[0])
                #to_delete.append(df[i][~df[i].isnull()].head(1).index.to_list()[0])

    str_index = [i.upper() for i in str_index]
    df.columns = str_index

    return type_index

In [392]:
#main
#path = r'./reto/1.PNG'
path = r'.\reto\2.PNG'
image = cv2.imread(path)
gray = get_grayscale(image)
th = thresholding(gray)
openingg = opening(gray)
cannyy = canny(gray)
img = image
hImg, wImg, _ = img.shape

custom_config = r'-c tessedit_char_blacklist=$¢§_{} --oem 3 --psm 4' 
data = pytesseract.image_to_data(img, lang= 'eng+spa', config=custom_config, output_type= 'dict')


In [393]:
pd.DataFrame(data).tail(50)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text
236,5,1,25,2,16,5,403,646,35,13,96.824066,que
237,5,1,25,2,16,6,445,640,95,16,96.386604,continúan
238,5,1,25,2,16,7,847,637,122,26,91.121750,"208,440,342"
239,5,1,25,2,16,8,1060,642,122,17,90.096397,"335,103,503"
240,4,1,25,2,17,0,62,684,1120,18,-1,
241,5,1,25,2,17,1,62,685,93,17,96.492119,Impuesto
242,5,1,25,2,17,2,161,684,23,15,96.883102,de
243,5,1,25,2,17,3,191,686,50,13,96.588425,renta
244,5,1,25,2,17,4,248,689,11,13,96.690979,y
245,5,1,25,2,17,5,265,684,167,18,96.804161,complementarios


In [394]:
indice_words = []
for i in range(len(data['text'])):
    if data['text'][i] != '' and data['text'][i] != ' ' and float(data['conf'][i]) >= 50 and data["text"][i] != "-" and data["text"][i] != "--" and data["text"][i] != "s" and data["text"] != "S":
        indice_words.append(i)

#print(len(indice_words))
#print(indice_words)
intervalo = calcular_espacios(data, indice_words)
print(intervalo)
coordenadas,data_organizer = coordenadas_extraccion(data, indice_words, intervalo)
df_data_organizer = pd.DataFrame(data_organizer)

(0, 14)


In [395]:
df_data_organizer.head(10)

,text,line,index,coordinates
0,Openatlas S.A.S.,0,"[4, 5]","(544, 702)"
1,NIT 900.465.720-5,1,"[7, 8]","(531, 715)"
2,Estado de Resultado Integral comparativo a 31 ...,2,"[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]","(350, 895)"
3,(Expresados en Pesos Colombianos),3,"[21, 22, 23, 24]","(470, 776)"
4,Años terminados el 31 de diciembre de:,4,"[120, 121, 122, 123, 124, 125, 126]","(61, 398)"
5,NOTAS,4,[127],"(586, 652)"
6,2020,4,[128],"(860, 919)"
7,2019,4,[129],"(1077, 1128)"
8,Ingresos de actividades ordinarias,5,"[132, 133, 134, 135]","(62, 350)"
9,13,5,[136],"(608, 630)"


In [396]:
df_data_organizer.loc[:,"coordinates"].apply(lambda x: (x[0]+x[1])/2).head(10)

0     623.0
1     623.0
2     622.5
3     623.0
4     229.5
5     619.0
6     889.5
7    1102.5
8     206.0
9     619.0
Name: coordinates, dtype: float64

In [397]:
del_headEnd(df_data_organizer)

In [398]:
df_data_organizer.head(5)

,text,line,index,coordinates
0,Años terminados el 31 de diciembre de:,4,"[120, 121, 122, 123, 124, 125, 126]","(61, 398)"
1,NOTAS,4,[127],"(586, 652)"
2,2020,4,[128],"(860, 919)"
3,2019,4,[129],"(1077, 1128)"
4,Ingresos de actividades ordinarias,5,"[132, 133, 134, 135]","(62, 350)"


In [399]:

#print("numero de posiciones: " + str(len(coordenadas)))
#del_headEnd(df_data_organizer)
min_max_h,colums = extracion_colums(df_data_organizer)

#Dibujado de los rectangulos en la imagen
for p in min_max_h:
    cv2.rectangle(img, (p[0], hImg-5),(p[1], 5), (random.randint(0,255), random.randint(0,255), random.randint(0,255)), 1)

for p in coordenadas:
    cv2.rectangle(img, (p[0], p[1]),(p[2], p[3]), (50, 50, 255), 1)

cv2.imwrite('salida.png',img)

df = pd.DataFrame(build_df(df_data_organizer , colums))

for i in df.columns:
    df[i] = df.apply(lambda x: x[i].replace(',','') if type(x[i]) == str else x[i], axis=1)
    df[i] = df.apply(lambda x: x[i].replace('.','') if type(x[i]) == str else x[i], axis=1)
    df[i] = df.apply(lambda x: '0' if x[i] == 'o' else x[i], axis=1)
#isNumericColum(df)

df.to_excel("data.xlsx", index=False)

"""
img = cv2.resize(img, (600, 700))
cv2.imshow('Result', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
"""

el tamaño de cl es:  9
(58, 550) [0, 4, 15, 23, 30, 38, 49, 52, 55, 58, 61, 65, 68, 71, 41, 8, 12, 19, 26, 34, 45]
(586, 652) [1]
(860, 919) [2]
(1077, 1128) [3]
(607, 630) [5, 9, 16, 20, 27, 31, 35, 42, 46, 62]
(829, 970) [6, 10, 13, 17, 24, 28, 32, 36, 39, 47, 50, 53, 59, 63, 66]
(1042, 1183) [7, 11, 14, 18, 22, 25, 29, 33, 37, 40, 44, 48, 51, 54, 60, 64, 67]
(956, 974) [21, 43, 56, 69, 72]
(1171, 1183) [57, 70, 73]
estos son a eliminar [[2], [3]]

[0, 4, 15, 23, 30, 38, 49, 52, 55, 58, 61, 65, 68, 71, 41, 8, 12, 19, 26, 34, 45]
[1, 5, 9, 16, 20, 27, 31, 35, 42, 46, 62]
[6, 10, 13, 17, 24, 28, 32, 36, 39, 47, 50, 53, 59, 63, 66, 2, 21, 43, 56, 69, 72]
[7, 11, 14, 18, 22, 25, 29, 33, 37, 40, 44, 48, 51, 54, 60, 64, 67, 3, 57, 70, 73]
la media de los len  18.5
[0, 4, 15, 23, 30, 38, 49, 52, 55, 58, 61, 65, 68, 71, 41, 8, 12, 19, 26, 34, 45]
[1, 5, 9, 16, 20, 27, 31, 35, 42, 46, 62]
[6, 10, 13, 17, 24, 28, 32, 36, 39, 47, 50, 53, 59, 63, 66, 2, 21, 43, 56, 69, 72]
[7, 11, 14, 18, 22, 25

"\nimg = cv2.resize(img, (600, 700))\ncv2.imshow('Result', img)\ncv2.waitKey(0)\ncv2.destroyAllWindows()\n"

In [400]:
type_index=[]                   
str_index=[]
counter_num = 1

for i in df.columns:
    size = df[i].apply(isNumeric).value_counts().sum()
    try:
        if 100*df[i].apply(isNumeric).value_counts()[True]/size >= 80:
            type_index.append('num')
            df[i] = df[i].apply(convert_parenthesis)
        else:
            type_index.append('text')
        
    except KeyError:
        type_index.append('text')

years = np.arange(2018,datetime.now().year + 1)
years = [str(i) for i in years]
years.append('NOTAS')
for i in df.columns:
    if type_index[i] == 'text':
        str_index.append(df[i][~df[i].isnull()].head(1).to_list()[0])
        #to_delete.append(df[i][~df[i].isnull()].head(1).index.to_list()[0])
    else:
        
        df_three = df[i][~df[i].isnull()].head(3).apply(lambda x: x if x.upper() in years else np.nan)
        if df_three.count() == 0:
            df_three = df[i][~df[i].isnull()].head(3).apply(isTime)
            if df_three.count() == 0:
                if len(df[i][~df[i].isnull()].head(3)[~df[i][~df[i].isnull()].apply(isNumeric).head(3)].to_list()) == 0:
                    str_index.append('num' + str(counter_num))
                    counter_num += 1
                    
                else:
                    str_index.append(df[i][~df[i].isnull()].head(3)[~df[i][~df[i].isnull()].apply(isNumeric).head(3)].to_list()[0])

            else:
                str_index.append(df_three[~df_three.isnull()].head(1).to_list()[0])
        else:
            str_index.append(df_three[~df_three.isnull()].head(1).to_list()[0])
            #to_delete.append(df[i][~df[i].isnull()].head(1).index.to_list()[0])

str_index = [i.upper() for i in str_index]

In [401]:
df.columns = str_index

In [402]:
df

,AÑOS TERMINADOS EL 31 DE DICIEMBRE DE:,NOTAS,2020,2019
4,Años terminados el 31 de diciembre de:,NOTAS,2020,2019
5,Ingresos de actividades ordinarias,13,2509807756,2188027466
6,Devoluciones,13,-152696992,-99541164
7,Utilidad bruta,NaN,2357110764,2088486302
8,Gastos de administración,14,-2090630317,-1651900012
9,Gastos de ventas,15,-10,-42541029
10,Resultado de actividades de la operación,NaN,266480447,394045261
11,Otros ingresos,16,25582912,978355
12,Gastos extraordinarios,17,-13447225,-12102171
13,Otros gastos,18,-506000,-14565791
